In [1]:
import pandas as pd
from functools import reduce

# Let's explore the dataset 

In [168]:
num_ref = pd.read_csv('Data/asylum_seekers/population.csv', skiprows=14)
#we are only interested in number of refugees
num_ref = num_ref[num_ref.columns[[0,1,2,5]]]
num_ref.head()

,Year,Country of origin,Country of origin (ISO),Refugees under UNHCR's mandate
0,1996,Afghanistan,AFG,2674234
1,1996,Albania,ALB,5784
2,1996,Algeria,DZA,2245
3,1996,Angola,AGO,249686
4,1996,Egypt,EGY,1198


In [177]:
pop = pd.read_csv('Data/Total_Population/Population.csv', skiprows=4, encoding = "ISO-8859-1")
#exclude years until 1996
rem_cols = [c for c in pop.columns if c[:2] == "19"][:-4]
pop.drop(columns = rem_cols, inplace = True)
#Drop indicator code and indicator name
rem_cols = [c for c in pop.columns if c[:3] == "Ind"]
pop.drop(columns = rem_cols, inplace = True)
#last two columns are NaN
pop.drop(columns = pop.columns[-2:], inplace = True)
#now convertcolumns into rows
pop = pop.melt(id_vars=["Country Name", "Country Code"], 
        var_name="Year", 
        value_name="Population")
pop = pop.sort_values(by=['Country Name', 'Year'])# we can sort by country or by year
pop.head()

,Country Name,Country Code,Year,Population
1,Afghanistan,AFG,1996,18853437.0
265,Afghanistan,AFG,1997,19357126.0
529,Afghanistan,AFG,1998,19737765.0
793,Afghanistan,AFG,1999,20170844.0
1057,Afghanistan,AFG,2000,20779953.0


In [233]:
#Here we will use only the number of deaths summed up over one year, Nan values are replaced with zero
disasters = pd.read_excel('Data/Disaster/disasters.xlsx', engine='openpyxl', skiprows = 6)
disasters['Country Code'] = [code[-3:] for code in disasters['Dis No']]
#move country code to the first position
cols = disasters.columns.tolist()
cols.insert(1, cols.pop(-1))# move it to the right position
disasters = disasters[cols]
#We sum all the disasters up for one year
#to get the best representation we will use total deaths and total affected since deaths are not included in that
tot_aff = disasters.groupby(['Country Code','Year'])['Total Affected'].sum().reset_index()
tot_deaths = disasters.groupby(['Country Code','Year'])['Total Deaths'].sum().reset_index()
disasters = pd.concat([tot_aff, tot_deaths], axis = 1)
#remove columns that occer twice
disasters = tot_aff.join(tot_deaths['Total Deaths'])
disasters.head()

,Country Code,Year,Total Affected,Total Deaths
0,AFG,1996,13230.0,130.0
1,AFG,1997,20830.0,229.0
2,AFG,1998,165836.0,7353.0
3,AFG,1999,113162.0,205.0
4,AFG,2000,2582268.0,634.0


In [ ]:
hdi = pd.read_csv('Data/HDI/HDI.csv', skiprows=5,encoding = "ISO-8859-1")
#Remove all unnamed columns
cols = [c for c in hdi.columns if c[:7] != 'Unnamed']
hdi = hdi[cols]
hdi.head()

In [142]:
precipitation = pd.read_csv('Data/Precipitation/Precipitation.csv', skiprows=4)
#precipitation.head() #almost only nan values, probably better to exclude or find new dataset

In [165]:
temp = pd.read_csv('Data/Temperature/Temp_noflags.csv',encoding = "ISO-8859-1")[:8398]#last entry that contains country info
#temperature change not absolute values
#exclude years before 1996
rem_cols = [c for c in temp.columns if c[:3] == "Y19"][:-4]
temp.drop(columns = rem_cols, inplace = True)
#Remove the Y in front of years
columns_old = temp.columns[7:]
columns_new = [c[1:] for c in temp.columns[7:]]
temp.rename(columns = dict(zip(columns_old, columns_new)),
          inplace=True, errors='raise')
#temp
temp['Months'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December',
       'Dec\x96Jan\x96Feb', 'Mar\x96Apr\x96May', 'Jun\x96Jul\x96Aug',
       'Sep\x96Oct\x96Nov', 'Meteorological year'], dtype=object)

In [141]:
wgi = pd.read_csv('Data/WGI/WGIData.csv')
#last column is unnamed and empty
wgi.drop(columns = wgi.columns[-1:], inplace = True)
#years 1997 and 2001 are missing, lets average over the following and preceding year
wgi['1997'] = wgi[['1996', '1998']].mean(axis = 1)
cols = wgi.columns.tolist()
cols.insert(5, cols.pop(-1))# move it to the right position
wgi = wgi[cols]
wgi['2001'] = wgi[['2000', '2002']].mean(axis = 1)
cols = wgi.columns.tolist()
cols.insert(8, cols.pop(-1))# move it to the right position
wgi = wgi[cols]
wgi.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1996,1997,1998,2000,2001,2002,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.236276,-1.180848,-1.295380,-1.279373,-1.263366,...,-1.635723,-1.579179,-1.419888,-1.436761,-1.354784,-1.341994,-1.526352,-1.517361,-1.496834,-1.401076
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,9.000000,9.000000,10.000000,11.000000,11.000000,11.000000,10.000000,10.000000,10.000000,10.000000
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,7.047445,9.793815,5.076142,5.063324,5.050505,...,0.952381,0.947867,2.369668,1.895735,5.288462,6.250000,3.365385,3.846154,4.326923,6.730769
3,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.480769,1.442308,0.000000,0.000000,0.000000,1.923077
4,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,27.419360,29.431330,31.443300,29.441620,30.629900,31.818180,...,3.809524,5.687204,11.848340,9.952606,12.500000,12.500000,9.615385,9.615385,9.615385,12.019230
